In [1]:
import numpy as np
import pandas as pd

In [2]:
def cal_phi(y):
    return(np.mean(ytrain==1))

def cal_mu1(x,y):
    m = len(y)
    y_pos = np.sum(y==0)
    conditional_sum_x = np.zeros_like(x.iloc[1,:], dtype=np.float64)
    for i in range(m):
        xi = x.iloc[i,:]
        yi = y[i]
        if (yi==1):
            conditional_sum_x += xi
        else:
            conditional_sum_x += np.zeros_like(xi)
    return (1/m)*conditional_sum_x/y_pos
    
def cal_mu0(x,y):
    m = len(y)
    y_neg = np.sum(y==1)
    conditional_sum_x = np.zeros_like(x.iloc[1,:])
    for i in range(m):
        xi = x.iloc[i,:]
        yi = y.iloc[i]
        if (yi==0):
            conditional_sum_x += xi
        else:
            conditional_sum_x += np.zeros_like(xi)
    return (1/m)*conditional_sum_x/y_neg
    
def cal_sigma(x, y, mu0, mu1):
    mu = [mu0, mu1]
    m = len(y)

    x_minus_mu = x.iloc[0] - mu[int(y[0])]
    x_minus_mu = x_minus_mu.values.reshape(*(x_minus_mu.shape), 1)
    s = np.matmul(x_minus_mu, x_minus_mu.T)
 
    for i in range(1, m):
        x_minus_mu = x.iloc[i] - mu[int(y[i])]
        x_minus_mu = x_minus_mu.values.reshape(*(x_minus_mu.shape), 1)
        s += np.matmul(x_minus_mu, x_minus_mu.T)
    s = s/m
    return(s)

def px_py(x, mu, sigma):
    pi = 3.1415926535
    dim = len([mu])
    expval = np.matmul((x-mu),np.linalg.inv(sigma))
    return ((1/(2*pi)**(dim/2)*np.sqrt(np.linalg.det(sigma)))*np.exp(-0.5*np.dot(expval,(x-mu).T)))

def cal_py(y, phi):
    if (y==1):
        return phi
    else:
        return 1-phi


In [5]:
dtrain1 = pd.read_csv("ds1_train.csv")
dtest1 = pd.read_csv("ds1_test.csv")

xtrain1 = dtrain[['x_1','x_2']]
ytrain1 = dtrain['y']
xtest1 = dtest[['x_1','x_2']]
ytest1 = dtest['y']
phi1 =  cal_phi(ytrain1)
mu1_1 = cal_mu1(xtrain1,ytrain1)
mu0_1 = cal_mu0(xtrain1,ytrain1)
sig1 = cal_sigma(xtrain1, ytrain1, mu0, mu1)

px_y0_1 = px_py(xtest1, mu0_1, sig1)*cal_py(0, phi1)
m = len(ytest1)
for i in range(m):
    if (px_y0_1[i] > 0.5):
        y_pred1[i]=0
    else:
        y_pred1[i]=1
print('accuracy -> {}'.format(sum(y_pred1 == ytest1)/ytest1.shape[0]))


NameError: name 'dtrain' is not defined

In [4]:
dtrain2 = pd.read_csv("ds2_train.csv")
dtest2 = pd.read_csv("ds2_test.csv")

xtrain2 = dtrain2[['x_1','x_2']]
ytrain2 = dtrain2['y']
xtest2 = dtest2[['x_1','x_2']]
ytest2 = dtest2['y']
phi2 =  cal_phi(ytrain2)
mu1_2 = cal_mu1(xtrain2,ytrain2)
mu0_2 = cal_mu0(xtrain2,ytrain2)
sig2 = cal_sigma(xtrain2, ytrain2, mu0_2, mu1_2)

px_y0_2 = px_py(xtest2, mu0_2, sig2)*cal_py(0, phi2)
m = len(ytest2)
for i in range(m):
    if (px_y0_2[i] > 0.5):
        y_pred2[i]=0
    else:
        y_pred2[i]=1
print('accuracy -> {}'.format(sum(y_pred2 == ytest2)/ytest2.shape[0]))


NameError: name 'ytrain' is not defined

In [ ]:
expval = np.matmul((xtest1-mu0_1),np.linalg.inv(sig))
np.dot(expval,(xtest1-mu0_1).T)

array([[0.38264832, 0.66220221, 0.46359646, ..., 0.48541294, 0.73550078,
        0.39997161],
       [0.66220221, 1.17437855, 0.91084258, ..., 1.0076553 , 1.38466805,
        0.85090744],
       [0.46359646, 0.91084258, 0.97678402, ..., 1.22905587, 1.31872978,
        1.09156248],
       ...,
       [0.48541294, 1.0076553 , 1.22905587, ..., 1.60543818, 1.59331599,
        1.44458838],
       [0.73550078, 1.38466805, 1.31872978, ..., 1.59331599, 1.85426446,
        1.39408441],
       [0.39997161, 0.85090744, 1.09156248, ..., 1.44458838, 1.39408441,
        1.30559775]])

In [ ]:
mu = [mu0,mu1]
s = xtr.loc[1,:]-mu[int(ytr[1])]
s 

x_1      3.771027
x_2    343.987218
dtype: float64

In [ ]:
px_y0_1

array([[65.81247502, 57.22738056, 63.20196162, ..., 62.51628624,
        55.16800573, 65.2448925 ],
       [57.22738056, 44.29821115, 50.53733057, ..., 48.14926727,
        39.87699316, 52.0747326 ],
       [63.20196162, 50.53733057, 48.89824776, ..., 43.10356196,
        41.21361564, 46.1710201 ],
       ...,
       [62.51628624, 48.14926727, 43.10356196, ..., 35.70941864,
        35.92651405, 38.69999161],
       [55.16800573, 39.87699316, 41.21361564, ..., 35.92651405,
        31.53195073, 39.68968649],
       [65.2448925 , 52.0747326 , 46.1710201 , ..., 38.69999161,
        39.68968649, 41.48511525]])

In [ ]:
m = len(y)
    mu = [mu0, mu1]
    x_minus_mu = x.iloc[1,:] - mu[int(y[1])]
    x_minus_mu = np.reshape(*(x_minus_mu.shape), 1)
    sqr_sum = np.matmul(x_minus_mu, x_minus_mu.T)
    for i in range(m):
        xi = x.iloc[i,:]
        yi = y[i]
        if (yi==1):
            sqr = (xi-mu1)*(xi-mu1)
        else:
            sqr = (xi-mu0)*(xi-mu0)
        sqr_sum += sqr
    return sqr_sum/m